<a href="https://colab.research.google.com/github/Shosh3sb/computer_vision_512/blob/main/classification_two.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Classification two:
A.

In [ ]:
import tensorflow as tr

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# Load  dataset
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()



170498071/170498071 [==============================] - 52s 0us/step


In [ ]:
print('x_train shape',x_train.shape)
print('y_train shape'y_train.shape)
print('x_test shape'x_test.shape)
print('y_test shape'y_test.shape)

In [ ]:
# Normalize pixel between 0 and 1
x_train = x_train.astype("float32") / 255
x_test = x_test.astype("float32") / 255

# Convert labels_to_one_hot encoding
num_classes = 10
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)



In [ ]:

# model
model = keras.Sequential(
    [
        layers.Conv2D(32, (3, 3), activation="relu", input_shape=(32, 32, 3)),
        layers.MaxPooling2D(pool_size=(2, 2)),

        layers.Conv2D(64, (3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),

        layers.Conv2D(128, (3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
    #flatten
        layers.Flatten(),
    #fully conected layer
        layers.Dense(num_classes, activation="softmax"),
    ]
)



In [ ]:
# Compile the model
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])


In [ ]:
# fit the model
batch_size = 64
epochs = 10
model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1)



Epoch 1/10
704/704 [==============================] - 51s 72ms/step - loss: 1.5978 - accuracy: 0.4188 - val_loss: 1.3373 - val_accuracy: 0.5222
Epoch 2/10
704/704 [==============================] - 51s 73ms/step - loss: 1.2396 - accuracy: 0.5659 - val_loss: 1.1189 - val_accuracy: 0.6134
Epoch 3/10
704/704 [==============================] - 51s 72ms/step - loss: 1.0839 - accuracy: 0.6240 - val_loss: 1.0533 - val_accuracy: 0.6382
Epoch 4/10
704/704 [==============================] - 53s 75ms/step - loss: 0.9796 - accuracy: 0.6624 - val_loss: 0.9925 - val_accuracy: 0.6544
Epoch 5/10
704/704 [==============================] - 53s 75ms/step - loss: 0.9053 - accuracy: 0.6903 - val_loss: 0.9761 - val_accuracy: 0.6640
Epoch 6/10
704/704 [==============================] - 50s 71ms/step - loss: 0.8463 - accuracy: 0.7100 - val_loss: 0.9639 - val_accuracy: 0.6702
Epoch 7/10
704/704 [==============================] - 50s 71ms/step - loss: 0.7915 - accuracy: 0.7285 - val_loss: 0.8860 - val_accuracy:

In [ ]:
# test the model on the test set
result = model.evaluate(x_test, y_test, verbose=0)
print("Test loss:", result[0])
print("Test accuracy:", result[1])

Test loss: 0.915831446647644
Test accuracy: 0.6909000277519226


B. Replace the convolution blocks with inception blocks and test performance

In [ ]:
# Define Inception block
def inception_block(inputs, filters):
    path1 = layers.Conv2D(filters=filters[0], kernel_size=1, activation='relu')(inputs)
    path1 = layers.Conv2D(filters=filters[1], kernel_size=3, padding='same', activation='relu')(path1)

    path2 = layers.Conv2D(filters=filters[2], kernel_size=1, activation='relu')(inputs)
    path2 = layers.Conv2D(filters=filters[3], kernel_size=5, padding='same', activation='relu')(path2)

    path3 = layers.MaxPooling2D(pool_size=(3,3), strides=(1,1), padding='same')(inputs)
    path3 = layers.Conv2D(filters=filters[4], kernel_size=1, activation='relu')(path3)

    output = layers.concatenate([path1, path2, path3], axis=-1)
    return output



In [ ]:
# Load CIFAR-10 dataset
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()

In [ ]:
# Normalize pixel values to be between 0 and 1
x_train = x_train.astype("float32") / 255
x_test = x_test.astype("float32") / 255

# Convert labels to one-hot encoding
num_classes = 10
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)



In [ ]:
# Define the CNN architecture with Inception blocks
inputs = keras.Input(shape=(32, 32, 3))

x = layers.Conv2D(filters=32, kernel_size=3, activation='relu')(inputs)
x = layers.Conv2D(filters=64, kernel_size=3, activation='relu')(x)
x = layers.MaxPooling2D(pool_size=(2, 2))(x)

x = inception_block(x, [16, 24, 32, 4, 8])
x = inception_block(x, [32, 32, 48, 8, 16])
x = layers.MaxPooling2D(pool_size=(2, 2))(x)

x = inception_block(x, [48, 64, 64, 16, 32])
x = inception_block(x, [64, 64, 96, 24, 48])
x = layers.GlobalAveragePooling2D()(x)

outputs = layers.Dense(num_classes, activation="softmax")(x)

model = keras.Model(inputs=inputs, outputs=outputs)



In [ ]:
# Compile the model
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])



In [ ]:
# Train the model
batch_size = 64
epochs = 10
model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1)

Epoch 1/10
704/704 [==============================] - 318s 446ms/step - loss: 1.7340 - accuracy: 0.3406 - val_loss: 1.5537 - val_accuracy: 0.4390
Epoch 2/10
704/704 [==============================] - 310s 440ms/step - loss: 1.3254 - accuracy: 0.5162 - val_loss: 1.1964 - val_accuracy: 0.5690
Epoch 3/10
704/704 [==============================] - 312s 443ms/step - loss: 1.1278 - accuracy: 0.5940 - val_loss: 0.9918 - val_accuracy: 0.6510
Epoch 4/10
704/704 [==============================] - 308s 437ms/step - loss: 0.9829 - accuracy: 0.6502 - val_loss: 1.0014 - val_accuracy: 0.6464
Epoch 5/10
704/704 [==============================] - 312s 444ms/step - loss: 0.8863 - accuracy: 0.6867 - val_loss: 0.8518 - val_accuracy: 0.7036
Epoch 6/10
704/704 [==============================] - 310s 440ms/step - loss: 0.8087 - accuracy: 0.7154 - val_loss: 0.8323 - val_accuracy: 0.7082
Epoch 7/10
704/704 [==============================] - 311s 442ms/step - loss: 0.7453 - accuracy: 0.7364 - val_loss: 0.7369 -

In [ ]:
# Evaluate the model on the test set
result = model.evaluate(x_test, y_test, verbose=0)
print("Test loss:", result[0])
print("Test accuracy:", result[1])

Test loss: 0.7191810607910156
Test accuracy: 0.7476999759674072


C. Replace the inception blocks with residual blocks and test performance.

In [ ]:
# Define Residual block
class ResidualBlock(layers.Layer):
    def __init__(self, filters, strides=1, activation="relu", **kwargs):
        super(ResidualBlock, self).__init__(**kwargs)
        self.filters = filters
        self.strides = strides
        self.activation = activation

        self.conv1 = layers.Conv2D(
            filters, kernel_size=3, strides=strides, padding="same", use_bias=False
        )
        self.bn1 = layers.BatchNormalization()
        self.relu = layers.Activation(activation)
        self.conv2 = layers.Conv2D(
            filters, kernel_size=3, strides=1, padding="same", use_bias=False
        )
        self.bn2 = layers.BatchNormalization()

        if strides > 1:
            self.downsample_conv = layers.Conv2D(
                filters, kernel_size=1, strides=strides, padding="same", use_bias=False
            )
            self.downsample_bn = layers.BatchNormalization()
        else:
            self.downsample_conv = None
            self.downsample_bn = None

    def call(self, inputs, training=None):
        residual = inputs

        x = self.conv1(inputs)
        x = self.bn1(x, training=training)
        x = self.relu(x)
        x = self.conv2(x)
        x = self.bn2(x, training=training)

        if self.downsample_conv is not None:
            residual = self.downsample_conv(inputs)
            residual = self.downsample_bn(residual, training=training)

        x = layers.add([x, residual])
        x = self.relu(x)
        return x



In [ ]:
# Load CIFAR-10 dataset
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()

In [ ]:
# Normalize pixel values to be between 0 and 1
x_train = x_train.astype("float32") / 255
x_test = x_test.astype("float32") / 255

# Convert labels to one-hot encoding
num_classes = 10
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)


In [ ]:
# Define the CNN architecture with Residual blocks
inputs = keras.Input(shape=(32, 32, 3))

x = layers.Conv2D(filters=64, kernel_size=3, strides=1, padding="same")(inputs)
x = layers.BatchNormalization()(x)
x = layers.Activation("relu")(x)

x = ResidualBlock(filters=64, strides=1)(x)
x = ResidualBlock(filters=64, strides=1)(x)
x = ResidualBlock(filters=64, strides=1)(x)

x = ResidualBlock(filters=128, strides=2)(x)
x = ResidualBlock(filters=128, strides=1)(x)
x = ResidualBlock(filters=128, strides=1)(x)

x = ResidualBlock(filters=256, strides=2)(x)
x = ResidualBlock(filters=256, strides=1)(x)
x = ResidualBlock(filters=256, strides=1)(x)

x = layers.GlobalAveragePooling2D()(x)

outputs = layers.Dense(num_classes, activation="softmax")(x)

model = keras.Model(inputs=inputs, outputs=outputs)



In [ ]:
# Compile the model
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])


In [ ]:
# Train the model
batch_size = 64
epochs = 5
model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1)

Epoch 1/5
704/704 [==============================] - 3486s 5s/step - loss: 1.3814 - accuracy: 0.5008 - val_loss: 1.8246 - val_accuracy: 0.4540
Epoch 2/5
704/704 [==============================] - 3315s 5s/step - loss: 0.9061 - accuracy: 0.6785 - val_loss: 1.2628 - val_accuracy: 0.6024
Epoch 3/5
704/704 [==============================] - 3352s 5s/step - loss: 0.6995 - accuracy: 0.7537 - val_loss: 0.9664 - val_accuracy: 0.6714
Epoch 4/5
704/704 [==============================] - 3368s 5s/step - loss: 0.5596 - accuracy: 0.8043 - val_loss: 0.9221 - val_accuracy: 0.7078
Epoch 5/5
704/704 [==============================] - 3395s 5s/step - loss: 0.4731 - accuracy: 0.8348 - val_loss: 0.7090 - val_accuracy: 0.7638


In [ ]:
# Evaluate the model on the test set
result = model.evaluate(x_test, y_test, verbose=0)
print("Test loss:", result[0])
print("Test accuracy:", result[1])

Test loss: 0.7418727278709412
Test accuracy: 0.7533000111579895
